<a href="https://colab.research.google.com/github/PanoEvJ/summarization_RLHF/blob/main/rlaif_rank_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch
!pip install -q datasets
!pip install -q openai

In [ ]:
import os
import torch
import openai
import random
import getpass

from datasets import load_dataset, Dataset as HFDataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [ ]:
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

### Set the Constitution

In [ ]:
constitution = """
```CONSTITUTION
Writing a good summary involves condensing a larger piece of text or content while preserving its key information and main points. Here are some principles to keep in mind when crafting an effective summary:

Understand the Material: Before summarizing, thoroughly read or engage with the material to ensure you grasp its main ideas, themes, and supporting details. This comprehension is crucial for creating an accurate summary.

Identify Key Points: Identify the most important and relevant information within the text. These are the ideas or facts that are essential for a reader to understand the content's core message.

Conciseness: Summaries should be concise and to the point. Eliminate unnecessary details, examples, or repetitions. Strive for clarity and brevity.

Maintain the Original's Structure: Try to preserve the original structure of the content, including the main ideas' logical flow. This helps maintain coherence and ensures your summary remains faithful to the original.

Use Your Own Words: Summarize in your own words rather than copying and pasting from the source. This demonstrates your understanding and avoids issues of plagiarism.

Highlight Main Ideas: Emphasize the primary concepts and arguments presented in the material. These are often found in topic sentences, headings, or concluding statements.

Avoid Personal Opinions: A summary should be objective and not include your personal opinions or interpretations. Stick to presenting the author's ideas.

Provide Context: Offer some context or background information when necessary to help readers understand the summarized content, especially if it's complex or unfamiliar.

Use Signal Phrases: Use phrases like "According to," "In summary," "The author argues," to introduce the author's ideas and maintain clarity about whose perspective is being summarized.

Check for Accuracy: Ensure that your summary accurately represents the original content. Avoid distorting or misrepresenting the author's ideas.

Review and Edit: After writing the summary, review it for clarity, grammar, and coherence. Ensure that it reads smoothly and is free of errors.
```
"""

prefix = """You are an expert in summarization. Your job it to rank summaries.
To rank the summaries follow the principles in the constitution given below in triple backtips.
You will be given one full text and two summaries for this text. You have to rank which is the better summary.
First, read both summaries. Then, denote the best as "chosen" and the other as "rejected".
Your output should be a simple dictionary:
{ "chosen" : x, "rejected": y}
, where x, y is 1, 2 or 2, 1 depending on the sequence that the two summaries are read.
\n
  """

system_prompt = f"{prefix, constitution}"

### Load the dataset

In [ ]:
summaries_dataset = load_dataset('PanoEvJ/T5_summarization_RLAIF', split='train')

In [ ]:
summaries_dataset[0]

### Rank the entire summary dataset

In [ ]:
def rank_summaries(example, **kwargs):

    ranked_summaries = {}

    full_text = example['prompt']
    summary_1 = example['summary_1']
    summary_2 = example['summary_2']

    summaries = f"""\n FULL TEXT: {full_text}
    \n SUMMARY 1: \n {summary_1}
    \n SUMMARY 2: \n {summary_2}

    Now rank the two summaries as instructed and using the given constitution:
    """

    temperature   = kwargs['temperature']
    model         = kwargs['model']
    system_prompt = kwargs['system_prompt']


    try:
      response = openai.ChatCompletion.create(
          temperature = temperature,
          model=model,
          messages=[{"role": "system", "content": system_prompt},
          {"role": "user", "content": summaries}],
          request_timeout=60000
      )
      response = response['choices'][0]['message']['content']
      assert isinstance(response, dict)  == True
      for key in response.keys():
          assert isinstance(key, int) == True
    except:
        c   = random.randint(1,2)
        if c == 1:
          r = 2
        else:
          r = 1
        response = {'chosen': c, 'rejected': r}

    ranked_summaries['chosen']   = example['summary_' + str(response['chosen'])]
    ranked_summaries['rejected'] = example['summary_' + str(response['rejected'])]

    return ranked_summaries

In [ ]:
fn_kwargs = {
    "temperature": 0.,
    "model": "gpt-3.5-turbo",
    "system_prompt": system_prompt,
}

summaries_dataset = summaries_dataset.map(rank_summaries, fn_kwargs=fn_kwargs, batched=False)

In [ ]:
preference_dataset = summaries_dataset.remove_columns(["summary_1", "summary_2"])
preference_dataset[0]

In [ ]:
hf_token = # ENTER YOUR HUGGINGFACE TOKEN HERE

In [ ]:
preference_dataset.push_to_hub('PanoEvJ/GPT3.5_summarization_preference_RLAIF', token=hf_token)